In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv('mnist_test.csv')

In [2]:
data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T #transpose
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 230.##

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 230


In [26]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(0, Z)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [27]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [33]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 250)

Iteration:  0
[6 9 3 ... 9 6 6] [7 1 1 ... 3 1 5]
0.12788888888888889
Iteration:  10
[7 9 1 ... 3 2 6] [7 1 1 ... 3 1 5]
0.24022222222222223
Iteration:  20
[7 9 1 ... 3 8 6] [7 1 1 ... 3 1 5]
0.33555555555555555
Iteration:  30
[7 1 1 ... 3 1 6] [7 1 1 ... 3 1 5]
0.37277777777777776
Iteration:  40
[7 1 1 ... 3 1 6] [7 1 1 ... 3 1 5]
0.40444444444444444
Iteration:  50
[7 1 1 ... 3 8 6] [7 1 1 ... 3 1 5]
0.44477777777777777
Iteration:  60
[7 1 1 ... 3 8 6] [7 1 1 ... 3 1 5]
0.4801111111111111
Iteration:  70
[7 1 1 ... 8 8 6] [7 1 1 ... 3 1 5]
0.509
Iteration:  80
[7 1 1 ... 8 8 6] [7 1 1 ... 3 1 5]
0.5267777777777778
Iteration:  90
[7 1 1 ... 8 8 6] [7 1 1 ... 3 1 5]
0.5461111111111111
Iteration:  100
[7 1 1 ... 8 8 6] [7 1 1 ... 3 1 5]
0.5677777777777778
Iteration:  110
[7 1 1 ... 8 1 6] [7 1 1 ... 3 1 5]
0.5925555555555555
Iteration:  120
[7 1 1 ... 8 1 6] [7 1 1 ... 3 1 5]
0.6182222222222222
Iteration:  130
[7 1 1 ... 8 1 6] [7 1 1 ... 3 1 5]
0.6434444444444445
Iteration:  140
[7 1 1 .